In [1]:
from sympy import Function, Symbol, E, Integral, diff, limit, sqrt, simplify, series, pi, oo, erf, cosh
from sympy.stats import Normal, cdf

In [2]:
delta = Symbol('Delta', positive=True)
tau = Symbol('tau', positive=True)
sig = Symbol('sigma', positive=True)
theta = Symbol('theta', positive=True)

In [3]:
# tick width
delta

Delta

In [4]:
# rebalancing period
tau

tau

In [5]:
# log-normal vol
sig

sigma

In [6]:
# pool fee volume per unit of virtual reserves
theta

theta

In [7]:
# standard normal RV
Z = Normal('Z', 0, 1)
Z

Z

In [8]:
# alias standard norm cdf: Phi(z)
Phi = cdf(Z)

In [9]:
# "normalized" delta
d = delta / (sig * sqrt(tau))
d

Delta/(sigma*sqrt(tau))

In [10]:
# term for fee revenue EV unevaluated
t = Symbol('t')
d_t = delta / (sig * sqrt(t))
integrand_fee = Phi(d_t) - Phi(-d_t) + E**(sig**2 * tau / 2) * ( Phi(d_t - sig * sqrt(t)) - Phi(-d_t - sig * sqrt(t)) )

ev_fee_int = theta * Integral(integrand_fee, (t, 0, tau))
ev_fee_int

theta*Integral((-erf(sqrt(2)*(-Delta/(sigma*sqrt(t)) - sigma*sqrt(t))/2)/2 + erf(sqrt(2)*(Delta/(sigma*sqrt(t)) - sigma*sqrt(t))/2)/2)*exp(sigma**2*tau/2) + erf(sqrt(2)*Delta/(2*sigma*sqrt(t))), (t, 0, tau))

In [11]:
# terms for fee revenue EV -- integral evaluated with Wolfram
ev_fee_term00 = (1/(4 * sig**2)) * ( 2 * (1 - cosh(2*delta)) +  (2 * delta + 1 - E**(-2*delta)) * erf((-d + sig * sqrt(tau)) / sqrt(2)) + (2*delta - 1 + E**(2 * delta)) * erf( (d + sig * sqrt(tau)) /sqrt(2)) )
ev_fee_term00

((2*Delta + 1 - exp(-2*Delta))*erf(sqrt(2)*(-Delta/(sigma*sqrt(tau)) + sigma*sqrt(tau))/2) + (2*Delta + exp(2*Delta) - 1)*erf(sqrt(2)*(Delta/(sigma*sqrt(tau)) + sigma*sqrt(tau))/2) - 2*cosh(2*Delta) + 2)/(4*sigma**2)

In [12]:
ev_fee_term01 = (tau / 2) * (erf( (d - sig * sqrt(tau)) / sqrt(2)) + erf( (d + sig * sqrt(tau)) / sqrt(2)))
ev_fee_term01

tau*(erf(sqrt(2)*(Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau))/2) + erf(sqrt(2)*(Delta/(sigma*sqrt(tau)) + sigma*sqrt(tau))/2))/2

In [13]:
ev_fee_term12 = sqrt(tau/(2 * pi)) * (1/sig) * (E**(-(d + sig * sqrt(tau))**2 / 2) - E**(-(d - sig * sqrt(tau))**2 / 2))
ev_fee_term12

sqrt(2)*sqrt(tau)*(exp(-(Delta/(sigma*sqrt(tau)) + sigma*sqrt(tau))**2/2) - exp(-(Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau))**2/2))/(2*sqrt(pi)*sigma)

In [14]:
ev_fee_base = ( (delta / sig)**2 + tau ) * erf(delta / (sig * sqrt(2 * tau))) + sqrt(2/pi) * sqrt(tau) * (delta/sig) * E**(-(delta/(sig * sqrt(tau)))**2/2) - (delta / sig)**2
ev_fee_base

-Delta**2/sigma**2 + sqrt(2)*Delta*sqrt(tau)*exp(-Delta**2/(2*sigma**2*tau))/(sqrt(pi)*sigma) + (Delta**2/sigma**2 + tau)*erf(sqrt(2)*Delta/(2*sigma*sqrt(tau)))

In [15]:
ev_fee = theta * ( ev_fee_base + E**(sig**2 * tau / 2) * (ev_fee_term00 + ev_fee_term01 + ev_fee_term12) )
ev_fee

theta*(-Delta**2/sigma**2 + sqrt(2)*Delta*sqrt(tau)*exp(-Delta**2/(2*sigma**2*tau))/(sqrt(pi)*sigma) + (Delta**2/sigma**2 + tau)*erf(sqrt(2)*Delta/(2*sigma*sqrt(tau))) + (tau*(erf(sqrt(2)*(Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau))/2) + erf(sqrt(2)*(Delta/(sigma*sqrt(tau)) + sigma*sqrt(tau))/2))/2 + sqrt(2)*sqrt(tau)*(exp(-(Delta/(sigma*sqrt(tau)) + sigma*sqrt(tau))**2/2) - exp(-(Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau))**2/2))/(2*sqrt(pi)*sigma) + ((2*Delta + 1 - exp(-2*Delta))*erf(sqrt(2)*(-Delta/(sigma*sqrt(tau)) + sigma*sqrt(tau))/2) + (2*Delta + exp(2*Delta) - 1)*erf(sqrt(2)*(Delta/(sigma*sqrt(tau)) + sigma*sqrt(tau))/2) - 2*cosh(2*Delta) + 2)/(4*sigma**2))*exp(sigma**2*tau/2))

In [16]:
# check on wolfram (1) ..
simplify(diff(ev_fee_base, tau, 1))

erf(sqrt(2)*Delta/(2*sigma*sqrt(tau)))

In [17]:
# check on wolfram (2) ..
simplify(diff(ev_fee_term00 + ev_fee_term01 + ev_fee_term12, tau, 1))

erf(sqrt(2)*(Delta - sigma**2*tau)/(2*sigma*sqrt(tau)))/2 + erf(sqrt(2)*(Delta + sigma**2*tau)/(2*sigma*sqrt(tau)))/2

In [18]:
# term for principal EV
ev_principal = 2 * E**(sig**2 * tau / 8) * (Phi(d - sig * sqrt(tau) / 2) - Phi(-d - sig * sqrt(tau) / 2)) - E**(-delta / 2) * (1 - Phi(-d) + E**(sig**2 * tau / 2) * Phi(d - sig * sqrt(tau))) + E**(delta/2) * (1 - Phi(d) + E**(sig**2 * tau/2) * Phi(-d - sig*sqrt(tau)))
ev_principal

2*(-erf(sqrt(2)*(-Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau)/2)/2)/2 + erf(sqrt(2)*(Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau)/2)/2)/2)*exp(sigma**2*tau/8) + ((erf(sqrt(2)*(-Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau))/2)/2 + 1/2)*exp(sigma**2*tau/2) - erf(sqrt(2)*Delta/(2*sigma*sqrt(tau)))/2 + 1/2)*exp(Delta/2) - ((erf(sqrt(2)*(Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau))/2)/2 + 1/2)*exp(sigma**2*tau/2) + erf(sqrt(2)*Delta/(2*sigma*sqrt(tau)))/2 + 1/2)*exp(-Delta/2)

In [19]:
# intermediate function to make life easier
# NOTE: g = ev_fee + ev_principal
g = ev_fee + ev_principal
g

theta*(-Delta**2/sigma**2 + sqrt(2)*Delta*sqrt(tau)*exp(-Delta**2/(2*sigma**2*tau))/(sqrt(pi)*sigma) + (Delta**2/sigma**2 + tau)*erf(sqrt(2)*Delta/(2*sigma*sqrt(tau))) + (tau*(erf(sqrt(2)*(Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau))/2) + erf(sqrt(2)*(Delta/(sigma*sqrt(tau)) + sigma*sqrt(tau))/2))/2 + sqrt(2)*sqrt(tau)*(exp(-(Delta/(sigma*sqrt(tau)) + sigma*sqrt(tau))**2/2) - exp(-(Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau))**2/2))/(2*sqrt(pi)*sigma) + ((2*Delta + 1 - exp(-2*Delta))*erf(sqrt(2)*(-Delta/(sigma*sqrt(tau)) + sigma*sqrt(tau))/2) + (2*Delta + exp(2*Delta) - 1)*erf(sqrt(2)*(Delta/(sigma*sqrt(tau)) + sigma*sqrt(tau))/2) - 2*cosh(2*Delta) + 2)/(4*sigma**2))*exp(sigma**2*tau/2)) + 2*(-erf(sqrt(2)*(-Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau)/2)/2)/2 + erf(sqrt(2)*(Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau)/2)/2)/2)*exp(sigma**2*tau/8) + ((erf(sqrt(2)*(-Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau))/2)/2 + 1/2)*exp(sigma**2*tau/2) - erf(sqrt(2)*Delta/(2*sigma*sqrt(tau)))/2 + 1/2)*exp(Delt

In [20]:
# full ev expression for uni v3 LPs
h = - ( 1 + E**(sig**2 * tau / 2) ) + (g / (1 - E**(-delta/2)))
h

-exp(sigma**2*tau/2) - 1 + (theta*(-Delta**2/sigma**2 + sqrt(2)*Delta*sqrt(tau)*exp(-Delta**2/(2*sigma**2*tau))/(sqrt(pi)*sigma) + (Delta**2/sigma**2 + tau)*erf(sqrt(2)*Delta/(2*sigma*sqrt(tau))) + (tau*(erf(sqrt(2)*(Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau))/2) + erf(sqrt(2)*(Delta/(sigma*sqrt(tau)) + sigma*sqrt(tau))/2))/2 + sqrt(2)*sqrt(tau)*(exp(-(Delta/(sigma*sqrt(tau)) + sigma*sqrt(tau))**2/2) - exp(-(Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau))**2/2))/(2*sqrt(pi)*sigma) + ((2*Delta + 1 - exp(-2*Delta))*erf(sqrt(2)*(-Delta/(sigma*sqrt(tau)) + sigma*sqrt(tau))/2) + (2*Delta + exp(2*Delta) - 1)*erf(sqrt(2)*(Delta/(sigma*sqrt(tau)) + sigma*sqrt(tau))/2) - 2*cosh(2*Delta) + 2)/(4*sigma**2))*exp(sigma**2*tau/2)) + 2*(-erf(sqrt(2)*(-Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau)/2)/2)/2 + erf(sqrt(2)*(Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau)/2)/2)/2)*exp(sigma**2*tau/8) + ((erf(sqrt(2)*(-Delta/(sigma*sqrt(tau)) - sigma*sqrt(tau))/2)/2 + 1/2)*exp(sigma**2*tau/2) - erf(sqrt(2)*Delta/(2*sigma*s

In [21]:
g_fn = Function('g')(delta, tau)
h_fn = - ( 1 + E**(sig**2 * tau / 2) ) + (g_fn / (1 - E**(-delta/2)))

In [22]:
h_fn

-exp(sigma**2*tau/2) - 1 + g(Delta, tau)/(1 - exp(-Delta/2))

In [23]:
# delta critical
diff(h_fn, delta, 1)

Derivative(g(Delta, tau), Delta)/(1 - exp(-Delta/2)) - g(Delta, tau)*exp(-Delta/2)/(2*(1 - exp(-Delta/2))**2)

In [24]:
# tau critical
diff(h_fn, tau, 1)

-sigma**2*exp(sigma**2*tau/2)/2 + Derivative(g(Delta, tau), tau)/(1 - exp(-Delta/2))

In [25]:
# d^2 delta
diff(h_fn, delta, 2)

(Derivative(g(Delta, tau), (Delta, 2)) + g(Delta, tau)*exp(-Delta/2)/(4*(1 - exp(-Delta/2))) - exp(-Delta/2)*Derivative(g(Delta, tau), Delta)/(1 - exp(-Delta/2)) + g(Delta, tau)*exp(-Delta)/(2*(1 - exp(-Delta/2))**2))/(1 - exp(-Delta/2))

In [26]:
# d^2 tau
diff(h_fn, tau, 2)

-sigma**4*exp(sigma**2*tau/2)/4 + Derivative(g(Delta, tau), (tau, 2))/(1 - exp(-Delta/2))

In [27]:
# dtau ddelta
diff(diff(h_fn, delta, 1), tau, 1)

Derivative(g(Delta, tau), Delta, tau)/(1 - exp(-Delta/2)) - exp(-Delta/2)*Derivative(g(Delta, tau), tau)/(2*(1 - exp(-Delta/2))**2)

### Expansion about $(0, 0)$

For first order expansion about $(\Delta, \tau) \to (0, 0)$, need the following terms at $(0, 0)$:

- $\partial_{\Delta} g$
- $\partial_{\tau} g$
- $\partial^2_{\tau} g$
- $\partial^2_{\Delta} g$
- $\partial_{\tau} \partial_{\Delta} g$

Split into principal and fee EV terms.

### Principal derivative terms

In [28]:
# zeroth order at (0, tau)
limit(ev_principal, delta, 0)

0

In [29]:
# zeroth order at (delta, 0)
limit(ev_principal, tau, 0)

(2*exp(Delta/2) - 2)*exp(-Delta/2)

In [30]:
# zeroth order at (0, 0)
limit(limit(ev_principal, tau, 0), delta, 0)

0

In [31]:
# first order wrt delta at (0, tau)
limit(diff(ev_principal, delta, 1), delta, 0)

-exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2)/2 + exp(sigma**2*tau/2)/2 + 1/2

In [32]:
# first order wrt delta at (0, 0)
limit(limit(diff(ev_principal, delta, 1), delta, 0), tau, 0)

1

In [33]:
# second order wrt delta at (0, tau)
limit(diff(ev_principal, delta, 1), delta, 0)

-exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2)/2 + exp(sigma**2*tau/2)/2 + 1/2

In [34]:
# third order wrt delta at (0, tau)
simplify(limit(diff(ev_principal, delta, 3), delta, 0))

-exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2)/8 + exp(sigma**2*tau/2)/8 + 1/8 - sqrt(2)/(2*sqrt(pi)*sigma*sqrt(tau))

In [35]:
# first order h wrt delta at (0, tau)
simplify(limit(diff(- ( 1 + E**(sig**2 * tau / 2) ) + (ev_principal / (1 - E**(-delta/2))), delta, 1), delta, 0))

-exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2)/4 + exp(sigma**2*tau/2)/4 + 1/4

In [36]:
# first order h wrt delta at (0, 0)
limit(simplify(limit(diff(- ( 1 + E**(sig**2 * tau / 2) ) + (ev_principal / (1 - E**(-delta/2))), delta, 1), delta, 0)), tau, 0)

1/2

In [37]:
# second order h wrt delta at (0, tau)
simplify(limit(diff(- ( 1 + E**(sig**2 * tau / 2) ) + (ev_principal / (1 - E**(-delta/2))), delta, 2), delta, 0))

-exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2)/8 + exp(sigma**2*tau/2)/8 + 1/8 - sqrt(2)/(3*sqrt(pi)*sigma*sqrt(tau))

In [38]:
# second order h wrt delta at (0, 0)
limit(simplify(limit(diff(- ( 1 + E**(sig**2 * tau / 2) ) + (ev_principal / (1 - E**(-delta/2))), delta, 2), delta, 0)), tau, 0)

-oo

In [39]:
# first order wrt tau at (delta, 0)
limit(diff(ev_principal, tau, 1), tau, 0)

(sigma**2*exp(Delta/2) - 2*sigma**2)*exp(-Delta/2)/4

In [40]:
# first order wrt tau at (0, 0)
limit(limit(diff(ev_principal, tau, 1), tau, 0), delta, 0)

-sigma**2/4

In [41]:
# second order wrt tau at (delta, 0)
limit(diff(ev_principal, tau, 2), tau, 0)

(sigma**4*exp(Delta/2) - 8*sigma**4)*exp(-Delta/2)/32

In [42]:
# second order wrt tau at (0, 0)
limit(limit(diff(ev_principal, tau, 2), tau, 0), delta, 0)

-7*sigma**4/32

In [43]:
# dtau, ddelta at (delta, 0)
limit(diff(diff(ev_principal, delta, 1), tau, 1), tau, 0)

sigma**2*exp(-Delta/2)/4

In [44]:
# first order h wrt tau at (delta, 0)
simplify(limit(diff(- ( 1 + E**(sig**2 * tau / 2) ) + (ev_principal / (1 - E**(-delta/2))), tau, 1), tau, 0))

sigma**2*exp(Delta/2)/(4*(1 - exp(Delta/2)))

In [45]:
# second order h wrt tau at (delta, 0)
simplify(limit(diff(- ( 1 + E**(sig**2 * tau / 2) ) + (ev_principal / (1 - E**(-delta/2))), tau, 2), tau, 0))

7*sigma**4*exp(Delta/2)/(32*(1 - exp(Delta/2)))

### Fee derivative terms

In [46]:
# zeroth order at (0, tau)
limit(ev_fee, delta, 0)

0

In [47]:
# zeroth order at (delta, 0)
limit(ev_fee, tau, 0)

0

In [48]:
# first order wrt delta at (0, tau)
limit(diff(ev_fee, delta, 1), delta, 0)

theta*(2*sqrt(2)*sigma*sqrt(tau) + 2*sqrt(pi)*exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2))/(sqrt(pi)*sigma**2)

In [49]:
# second order wrt delta at (0, tau)
limit(diff(ev_fee, delta, 2), delta, 0)

theta*(-2*exp(sigma**2*tau/2) - 2)/sigma**2

In [50]:
# third order wrt delta at (0, tau)
simplify(limit(diff(ev_fee, delta, 3), delta, 0))

4*theta*exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2)/sigma**2 + 4*sqrt(2)*theta/(sqrt(pi)*sigma**3*sqrt(tau))

In [51]:
# first order wrt tau at (delta, 0)
limit(diff(ev_fee, tau, 1), tau, 0)

2*theta

In [52]:
# second order wrt tau at (delta, 0)
limit(diff(ev_fee, tau, 2), tau, 0)

sigma**2*theta

In [53]:
# mixed second order ddelta, dtau at (0, tau)
simplify(limit(diff(diff(ev_fee, tau, 1), delta, 1), delta, 0))

theta*exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2) + 2*sqrt(2)*theta/(sqrt(pi)*sigma*sqrt(tau))

In [54]:
# dtau, ddelta at (delta, 0)
simplify(limit(diff(diff(ev_fee, tau, 1), delta, 1), tau, 0))

0

### Delta critical expressions

In [55]:
g1_delta0 = simplify(limit(diff(g, delta, 1), delta, 0))
g1_delta0

-exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2)/2 + exp(sigma**2*tau/2)/2 + 1/2 + 2*sqrt(2)*sqrt(tau)*theta/(sqrt(pi)*sigma) + 2*theta*exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2)/sigma**2

In [56]:
g2_delta0 = simplify(limit(diff(g, delta, 2), delta, 0))
g2_delta0

-2*theta*(exp(sigma**2*tau/2) + 1)/sigma**2

In [57]:
g3_delta0 = simplify(limit(diff(g, delta, 3), delta, 0))
g3_delta0

-exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2)/8 + exp(sigma**2*tau/2)/8 + 1/8 - sqrt(2)/(2*sqrt(pi)*sigma*sqrt(tau)) + 4*theta*exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2)/sigma**2 + 4*sqrt(2)*theta/(sqrt(pi)*sigma**3*sqrt(tau))

In [58]:
delta_c_denom = g1_delta0/24+ (5 * g2_delta0 / 4) + g3_delta0/3
delta_c_denom

-exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2)/16 + exp(sigma**2*tau/2)/16 + 1/16 + sqrt(2)*sqrt(tau)*theta/(12*sqrt(pi)*sigma) - sqrt(2)/(6*sqrt(pi)*sigma*sqrt(tau)) - 5*theta*(exp(sigma**2*tau/2) + 1)/(2*sigma**2) + 17*theta*exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2)/(12*sigma**2) + 4*sqrt(2)*theta/(3*sqrt(pi)*sigma**3*sqrt(tau))

In [59]:
delta_c_numerator = g2_delta0 / 2 - g1_delta0 / 4
delta_c_numerator

exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2)/8 - exp(sigma**2*tau/2)/8 - 1/8 - sqrt(2)*sqrt(tau)*theta/(2*sqrt(pi)*sigma) - theta*(exp(sigma**2*tau/2) + 1)/sigma**2 - theta*exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2)/(2*sigma**2)

#### Delta critical evaluated from TS expansion of $\partial_{\Delta} = 0$ eqn to third order

In [60]:
delta_c = simplify(delta_c_numerator / delta_c_denom)
delta_c

-6*pi*sigma*tau*(sqrt(pi)*sigma**2*(-exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2) + exp(sigma**2*tau/2) + 1) + 4*sqrt(2)*sigma*sqrt(tau)*theta + 4*sqrt(pi)*theta*(exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2) + 2*exp(sigma**2*tau/2) + 2))/(3*pi**(3/2)*sigma**3*tau*(-exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2) + exp(sigma**2*tau/2) + 1) + 4*sqrt(2)*pi*sigma**2*tau**(3/2)*theta - 8*sqrt(2)*pi*sigma**2*sqrt(tau) - 4*pi**(3/2)*sigma*tau*theta*(-17*exp(sigma**2*tau/2)*erf(sqrt(2)*sigma*sqrt(tau)/2) + 30*exp(sigma**2*tau/2) + 30) + 64*sqrt(2)*pi*sqrt(tau)*theta)

In [61]:
limit(delta_c, tau, 0)

0

In [62]:
limit(delta_c, tau, oo)

18/13

In [63]:
limit(delta_c, sig, oo)

18/13

In [69]:
E**(limit(delta_c, sig, oo)).evalf()

3.99328972875219